In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import json,os,re

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--incognito')
options.add_argument('--disable-popup-blocking')

driver = webdriver.Chrome(options = options)

listData = []
rentalList = []
stime = 2
ltime = 5
page = 1
maxPage = 66
decor = None
register = None

def visit():
    driver.get('https://business.591.com.tw/?type=1&kind=5')

def search():
    # change to the page of New Taipei City
    driver.find_element(By.CSS_SELECTOR, 'a.close').click()
    sltElement = driver.find_elements(By.CSS_SELECTOR, 'i.fa.fa-angle-down')
    sltElement[0].click()
    aElement = driver.find_element(By.CSS_SELECTOR, 'li.city-li a[data-id="3"]') #data-id="3": 新北市 
    aElement.click()
    inputElement = driver.find_element(By.CSS_SELECTOR, 'input.areamin.sh-info')
    inputElement.send_keys('10')
    sleep(stime)
    inputElement = driver.find_element(By.CSS_SELECTOR, 'input.areamax.sh-info')
    inputElement.send_keys('50')
    sleep(stime)
    btnElement = driver.find_element(By.CSS_SELECTOR, 'div.searchSum.last.area input.sh-btn')
    btnElement.click()
    sleep(stime)

def getMainData(): # 抓取列表的網址
    for page in range(maxPage):
        rentallist = driver.find_elements(By.CSS_SELECTOR,'ul.listInfo')
        for rental in rentallist:
            rent_id = rental.get_attribute('data-bind')
            listData.append({
                "id": rent_id,
            })
        sleep(ltime)
        nextPageElement = driver.find_element(By.CSS_SELECTOR, 'a.pageNext').click()
    print(listData)

def saveJson():
    file = open("rental_nt.json", "w", encoding="utf-8")
    file.write(json.dumps(listData, ensure_ascii=False))
    file.close()

def downLoadInfo():
    fp = open("rental_nt.json", "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()
    
    listResult = json.loads(strJson)
    print(listResult)
    for i in range(len(listResult)): # exception中斷時， 儲存資料後，改資料抓取起始點,如 (100,len(listResult))
        try:
            # 抓取原頁面資料
            driver.get("https://rent.591.com.tw/rent-detail-"+listResult[i]["id"]+".html")
            sleep(stime)
            if driver.find_elements(By.CSS_SELECTOR, "div.error-info") == []:  
                # 最上面地址資訊
                address = driver.find_element(By.CSS_SELECTOR, "span.addr").get_attribute("innerText")
                # 最右邊欄位資料
                price = driver.find_element(By.CSS_SELECTOR, "div.price").get_attribute("innerText")
                attr = driver.find_element(By.CSS_SELECTOR, "ul.attr").get_attribute("innerText")
                regexArea = r'[0-9]?[0-9]{2}'
                regexFloor = r'([\u4e00-\u9fa5]{2}|B?[0-9]?[0-9]F)/[1-9]?[0-9]F'
                area = re.search(regexArea,attr)[0]
                floor = re.search(regexFloor,attr)[0]
                # 中間欄位資料
                term = driver.find_elements(By.CSS_SELECTOR, "ul.clearfix.labelList li div.two em")
                deposit = term[0].get_attribute("innerText")
                fee = term[1].get_attribute("innerText")
                pklot = term[2].get_attribute("innerText")
                min_lease = term[3].get_attribute("innerText")
                decor = term[4].get_attribute("innerText")
                register = term[5].get_attribute("innerText")
                sleep(stime)
            else:
                continue
        except (KeyError,IndexError, TypeError):
            print(i+1)
    
        # 下拉卷軸，跑到網頁最底        
        js_code = f'''
            window.scrollTo({{
                top : { 4000 },
                behavior : "smooth"
            }})
        '''
        driver.execute_script(js_code)

        
        # 等待 google map頁面出現，抓取座標
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR,"div#mapRound>iframe.myframe"))
            )
            # 進入第一層 iframe
            iframe1 = driver.find_element(By.CSS_SELECTOR,"div#mapRound>iframe.myframe")
            driver.switch_to.frame(iframe1)
            sleep(stime)
            iframe2 = driver.find_element(By.CSS_SELECTOR,"div.propMapBarMap>iframe")
            g_location1 = iframe2.get_attribute("src")
            loc_regex = r"[0-9]?[0-9]{2}.[0-9]+,[0-9]?[0-9]{2}.[0-9]+"
            g_location1 = re.search(loc_regex,g_location1)[0]
            sleep(stime)
            # 進入第二層 iframe
            driver.switch_to.frame(iframe2)
            g_location2 = driver.find_element(By.CSS_SELECTOR, "div.place-name").get_attribute("innerText")
            g_address =  driver.find_element(By.CSS_SELECTOR, "div.address").get_attribute("innerText")
            sleep(stime)
            # 回到原頁面
            driver.switch_to.default_content()
        except (TimeoutException, NoSuchElementException):
            print("連線逾時")
            print(i+1)
            sleep(stime)
            
        # 加入資料到 rentalList[]
        rentalList.append({
            "no": i+1,
            "id": listResult[i]["id"],
            "address": address,
            "price": price,
            "area": area,
            "floor": floor,
            "deposit": deposit,
            "mgmt fee": fee,
            "pklot": pklot,
            "min_lease": min_lease,
            "decor": decor,
            "register": register,
            "g_location1": g_location1,
            "g_location2": g_location2,
            "g_address": g_address,
        })

def saveJsonFinal():
    file = open("rentalList_nt.json", "w", encoding="utf-8")
    file.write(json.dumps(rentalList, ensure_ascii=False))
    file.close()


def close():
    driver.quit()
        
if __name__ == "__main__":
    #visit()
    #search()
    #getMainData()
    #saveJson()
    downLoadInfo()
    saveJsonFinal()
    close()

In [ ]:
# 中斷時，確認筆數
len(rentalList)

In [ ]:
rentalList # 確認抓到哪一筆

In [ ]:
# 先儲存以抓下的資料，再更改 row 69的起始點
file = open("rentalList_nt15.json", "w", encoding="utf-8")
file.write(json.dumps(rentalList, ensure_ascii=False))
file.close()

In [ ]:
# 合併 JSON檔案
import json
import csv
totalData = []
for i in range(16):   
    fileName =f"rentalList_nt{i+1}.json"
    fp = open(fileName, "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()
    listResult = json.loads(strJson)
    for j in range(len(listResult)):
        totalData.append({
            "id": listResult[j]["id"],
            "address": listResult[j]["address"],
            "price": listResult[j]["price"],
            "area": listResult[j]["area"],
            "floor": listResult[j]["floor"],
            "deposit": listResult[j]["deposit"],
            "mgmt fee": listResult[j]["mgmt fee"],
            "pklot": listResult[j]["pklot"],
            "min_lease": listResult[j]["min_lease"],
            "decor": listResult[j]["decor"],
            "register": listResult[j]["register"],
            "g_location1": listResult[j]["g_location1"],
            "g_location2": listResult[j]["g_location2"],
            "g_address": listResult[j]["g_address"],
        })
len(totalData)

In [ ]:
# 儲存合併後的 新北市 JSON檔
file = open("rentalList_nt.json", "w", encoding="utf-8")
file.write(json.dumps(totalData, ensure_ascii=False))
file.close()